# Philippine electricity market


**Summary**

This article provides an **exploratory data analysis** on the Philippine electricity market by comparing the data provided by IEMOP and DOE on their websites.

## Data Import

The data comes from the published market participants list by IEMOP, the electricity market operator of the Philippines.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def get_data(url):
    html_data = pd.read_html(url)
    data = html_data[0]
    return data

In [3]:
df_wesm = get_data('http://www.iemop.ph/the-market/participants/wesm-members/')


In [4]:
df_retail = get_data('http://www.iemop.ph/the-market/participants/rcoa-tp/')

## Exploratory Data Analysis

We explore the dataset by looking at each column; looking at unique values, calculating summaries, and plotting distributions.

### Wholesale Electricity Spot Market

This section discusses the data on Wholesale Electricity Spot Market participants.

First, we look at the column names to see the variables from the data.

In [ ]:
df_wesm.columns

We get the counts of the unique values in each column.

In [ ]:
df_wesm.nunique()

#### Participant Name and Short Name

From the counts above, we notice that `PARTICIPANT NAME` and `SHORT NAME` are not equal. This means that there are duplicate values for these variables. We investigate by performing a cross-tabulation.

In [ ]:
df_cross = pd.crosstab(df_wesm['SHORT NAME'], df_wesm['PARTICIPANT NAME'],margins=True)
df_cross.tail()

Based on the tabulation, we can see that there are `PARTICIPANT NAME` with multiple `SHORT NAME`.

In [ ]:
df_cross.All[(df_cross.All > 1)].head(n=10)


We take a look at some of the values.

In [ ]:
df_wesm[df_wesm['SHORT NAME'] =='APRI']

In [ ]:
df_wesm[df_wesm['SHORT NAME'] =='AHC']

We can see that each `PARTICIPANT NAME` has multiple `RESOURCE` entries. From the inspected data above, we can infer that membership in the spot market is a disaggregation of the power plant units of each participating company.

#### Region

We look at the `REGION` variable and see that most of the participants are from Luzon.

In [ ]:
df_wesm.groupby('REGION')['RESOURCE'].nunique()


We look at the participants with dual regions.

In [ ]:
df_wesm[df_wesm['REGION'] =='LUZON / VISAYAS']

Except for the NGCP, which is a transmission company and NPC, all the participants with dual regions are Aggregators (WAG) with 'ceased' status. ERC has issued a circular to stop the operation of all WAGs.

#### Category

Just like any market, we can see that WESM participants are either buyers or sellers of electricity.

In [ ]:
df_wesm['CATEGORY'].unique()

#### Membership

Participants have either `DIRECT` or `INDIRECT` membership, which pertains to their grid connection status.

In [ ]:
df_wesm['MEMBERSHIP'].unique()

#### Effective Date

Date when they participated in the market. We convert the data type.

In [45]:
df_wesm = df_wesm.astype({'EFFECTIVE DATE': 'datetime64'})
df_wesm

,PARTICIPANT NAME,SHORT NAME,REGION,CATEGORY,MEMBERSHIP,RESOURCE,EFFECTIVE DATE,STATUS
0,GNPower Kauswagan Ltd. Co.,GNPKLCO,MINDANAO,GENERATOR,DIRECT,GNPK Coal Fired Power Plant Unit 4,2020-08-07,REGISTERED
1,GNPower Kauswagan Ltd. Co.,GNPKLCO,MINDANAO,GENERATOR,DIRECT,GNPK Coal Fired Power Plant Unit 3,2020-08-07,REGISTERED
2,FGP Corporation,FGP - San Lorenzo,LUZON,GENERATOR,DIRECT,San Lorenzo Natural Gas Power Plant 5,2006-06-26,REGISTERED
3,FGP Corporation,FGP - San Lorenzo,LUZON,GENERATOR,DIRECT,San Lorenzo Natural Gas Power Plant 6,2006-06-26,REGISTERED
4,First Gas Power Corporation,FGPCSTRA,LUZON,GENERATOR,DIRECT,Sta. Rita Natural Gas Power Plant 1,2006-06-26,REGISTERED
...,...,...,...,...,...,...,...,...
445,Philippine Power and Development Company,PPDC,LUZON,GENERATOR,DIRECT,Calibato Hydroelectric Power Plant,2020-05-30,REGISTERED
446,"Bataan 2020, Inc.",BATA02,LUZON,GENERATOR,DIRECT,Bataan 2020 Coal Fired Power Plant,2020-09-09,REGISTERED
447,Power Sector Assets & Liabilities Management C...,PSALM,MINDANAO,GENERATOR,DIRECT,255 MW Pulangi IV Hydroelectric Power Plant Un...,2020-10-03,REGISTERED
448,Power Sector Assets & Liabilities Management C...,PSALM,MINDANAO,GENERATOR,DIRECT,54.24 MW Mindanao I Geothermal Power Plant,2020-10-23,REGISTERED


In [ ]:
g1 = df_wesm.groupby(['REGION', 'CATEGORY', 'EFFECTIVE DATE', 'STATUS']).count().reset_index()
g1 = g1[g1['REGION']!='LUZON / VISAYAS']

#### Visualisations

In [ ]:
import plotly.express as px

fig = px.scatter(g1, x='EFFECTIVE DATE', y='RESOURCE', range_y=['0','25'], color = 'CATEGORY',facet_row="REGION")
fig.update_xaxes(rangeslider_visible=False)
fig.show()

In [90]:
names = df_wesm.apply(lambda col: str(col['PARTICIPANT NAME']) + ' ' + str(col['RESOURCE']), axis=1)

GNPower Kauswagan Ltd. Co. GNPK Coal Fired Power Plant Unit 4
GNPower Kauswagan Ltd. Co. GNPK Coal Fired Power Plant Unit 3
FGP Corporation San Lorenzo Natural Gas Power Plant 5
FGP Corporation San Lorenzo Natural Gas Power Plant 6
First Gas Power Corporation Sta. Rita Natural Gas Power Plant 1
First Gas Power Corporation Sta. Rita Natural Gas Power Plant 2
First Gas Power Corporation Sta. Rita Natural Gas Power Plant 3
First Gas Power Corporation Sta. Rita Natural Gas Power Plant 4
Power Sector Assets & Liabilities Management Corporation Kalayaan Hydro Electric Power Plant 1
Power Sector Assets & Liabilities Management Corporation Kalayaan Hydro Electric Power Plant 2
Power Sector Assets & Liabilities Management Corporation Kalayaan Hydro Electric Power Plant 3
Power Sector Assets & Liabilities Management Corporation Malaya Oil Thermal Power Plant 1
Power Sector Assets & Liabilities Management Corporation Casecnan Hydro Electric Power Plant
Power Sector Assets & Liabilities Management

In [91]:
import os
import gmaps
import googlemaps

#gmaps.configure(api_key=["GOOGLE_API_KEY"])

gmaps = googlemaps.Client(key=os.getenv('GOOGLE_API_KEY'))

In [92]:
# Geocoding an address
df_wesm_coded = pd.DataFrame({"PLACE":names[0:450]})
df_wesm_coded["LAT"] = None
df_wesm_coded["LON"] = None
geocode_result = []

for i in range(0, len(names[0:450]),1):
    geocode_result = gmaps.geocode(names[i])
    try:
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        lon = geocode_result[0]["geometry"]["location"]["lng"]
        df_wesm_coded.iat[i,df_wesm_coded.columns.get_loc("LAT")] = lat
        df_wesm_coded.iat[i,df_wesm_coded.columns.get_loc("LON")] = lon
    except Exception as e:
        lat = None
        lon = None
        #print('Error, skipping address...', e)


Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list index out of range
Error, skipping address... list

In [117]:
df_wesm_coded = df_wesm_coded[df_wesm_coded['LON'].notnull()]


In [121]:
import gmaps.datasets

gmaps.configure(api_key=os.getenv('GOOGLE_API_KEY'))


starbucks_df = df_wesm_coded[['LAT','LON']]

starbucks_layer = gmaps.symbol_layer(
    starbucks_df, fill_color='green', stroke_color='green', scale=2
)
fig = gmaps.figure(zoom_level=5, center=(12.8797, 121.7740))
fig.add_layer(starbucks_layer)
fig

Figure(layout=FigureLayout(height='420px'))

---

### Retail Market

This section discusses the Retail Market participants.

In [ ]:
df_retail.columns

In [ ]:
df_retail['CATEGORY'].unique()

In [ ]:
df_retail[df_retail['CATEGORY'] =='Contestable Customer'].head(n=20)